# Tensorflow

In [1]:
import tensorflow as tf

# Пример: классификация изображений MNIST

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

### Функции инициализаии переменных и создания слоев

In [4]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### Переводим входной вектор в 2D

In [5]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

### Сверточные слои и пулинг 

In [6]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [7]:
h_pool1.get_shape()[-1].value

32

In [8]:
W_conv2 = weight_variable([5, 5, h_pool1.get_shape()[-1].value, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [9]:
h_pool2.get_shape()

TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(64)])

### Полносвязный слой

In [10]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### Dropout

In [11]:
#TODO: добавить dropout после полносвязного слоя
keep_prob = tf.placeholder_with_default(
    tf.constant(1., tf.float32), [])
h_fc1 = tf.nn.dropout(x=h_fc1, keep_prob=keep_prob)

### Выходной слой

In [12]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2

### Функция потерь и оптимизатор

In [18]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_,
                                            logits=y_conv))

total_loss = cross_entropy + 0.1 * tf.nn.l2_loss(W_fc1)

train_step = tf.train.AdamOptimizer(1e-4).minimize(total_loss)

### Оценка качества

In [19]:
correct_prediction = tf.equal(
    tf.argmax(y_conv, 1),
    tf.argmax(y_, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction,
                                  tf.float32))

### Обучение

In [ ]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(1000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(
          feed_dict={x: batch[0],
                     y_: batch[1],
                    keep_prob: 1.})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0],
                              y_: batch[1],
                              keep_prob: 0.8})

  print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images, 
        y_: mnist.test.labels}))

### Визуализация в tensorboard

In [22]:
!rm -rf ./log/002/*

In [24]:
summary_train = tf.summary.merge([
    tf.summary.scalar('crossentropy-train', cross_entropy),
    tf.summary.scalar('accuracy-train', accuracy)])

summary_test = tf.summary.merge([
    tf.summary.scalar('crossentropy-test', cross_entropy),
    tf.summary.scalar('accuracy-test', accuracy)])


tf.summary.histogram

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('log/002', sess.graph)
    for i in range(100):
        if i % 10 == 0:
            summary, _, _ = sess.run([summary_test, 
                                      cross_entropy,
                                      accuracy], 
                            feed_dict={x: mnist.test.images,
                                       y_: mnist.test.labels,
                                      keep_prob: 1.})
            train_writer.add_summary(summary, i)
            # print('step %d, training accuracy %g' % 
            # (i, train_accuracy))
        batch = mnist.train.next_batch(50)
        
        train_step.run(feed_dict={x: batch[0],
                                  y_: batch[1],
                                  keep_prob: 0.9})
        
        summary, _, _ = sess.run([summary_train, 
                                   cross_entropy,
                                   accuracy], 
                                  feed_dict={x: batch[0],
                                         y_: batch[1],
                                         keep_prob: 1.})
        train_writer.add_summary(summary, i)

In [25]:
train_writer.close()